In [5]:
import numpy as np
import math


Risk Neutral Binomial Pricing

In [6]:
def up_move(sigma, time):
    #sigma is annual Volatility
    #time is the length of time in the model
    return exp(sigma * math.sqrt(time))

In [7]:
def down_move(up_move):
    #the down move factor is the inverse of the upmove factor
    return 1/up_move

In [8]:
def up_rn_prob(rate, time, down_move, up_move):
    #rate is the applicable risk free annual interest rate, continuously compounded
    #
    return (exp(rate * time)- down_move)/(up_move - down_move)